In [88]:
import pickle
import requests
import numpy as np
import time
import os
from tqdm.notebook import tqdm

In [8]:
with open(os.path.join("processed_data", "departments.pkl"), "rb") as f:
    departments = pickle.load(f)

dep_codes = sorted(departments.keys())
cities = [departments[c]["capital"] for c in dep_codes]
cities_coords = [departments[c]["capital_coords"] for c in dep_codes]
coords_list = []
for city in cities_coords:
    coords_list.append([city["lat"], city["lng"]])

In [36]:
len(cities_coords)

101

In [47]:
#url = "http://www.mapquestapi.com/directions/v2/routematrix"
url = "http://www.mapquestapi.com/directions/v2/route"
api_key = "mGe2Ac9v3ynZtrlCkWbN34DKhApIAVc1"
post_url = url + "?key=" + api_key

In [51]:
body = {
  "locations": [{"latLng": cc} for cc in cities_coords[:2]],
  "options": {
        "routeType": "fastest",
        "unit": "m",
  }
}

In [79]:
distance_matrix = -1 * np.ones((len(cities_coords), len(cities_coords)))
time_matrix = -1 * np.ones((len(cities_coords), len(cities_coords)))

for i in tqdm(range(len(cities_coords))):
    for j in range(len(cities_coords)):
        time.sleep(0.01)
        if i==j:
            distance_matrix[i, j] = 0
            time_matrix[i, j] = 0
            continue
        body["locations"] = [{"latLng": cities_coords[i]}, {"latLng": cities_coords[j]}]
        response = requests.post(post_url, json=body)
        if response.status_code != 200:
            continue
        rj = response.json()
        if "route" not in rj.keys():
            continue
        if "distance" not in rj["route"].keys() or "time" not in rj["route"].keys():
            continue
        distance_matrix[i, j] = rj["route"]["distance"]
        time_matrix[i, j] = rj["route"]["time"]
        
#response = requests.post(post_url, json=body)

In [80]:
np.save("distance_matrix.npy", distance_matrix)
np.save("time_matrix.npy", time_matrix)

In [84]:
counter = 0
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix)):
        if i == j:
            continue
        if distance_matrix[i,j] == 0:
            distance_matrix[i,j] = -2
        if time_matrix[i,j] == 0:
            time_matrix[i,j] = -2
            
print(counter)

0


In [85]:
distance_matrix

array([[  0.   , 316.097, 109.338, ...,  -1.   ,  -2.   ,  -2.   ],
       [317.307,   0.   , 276.597, ...,  -1.   ,  -2.   ,  -2.   ],
       [108.824, 273.914,   0.   , ...,  -1.   ,  -2.   ,  -2.   ],
       ...,
       [ -1.   ,  -1.   ,  -1.   , ...,   0.   ,  -1.   ,  -1.   ],
       [ -1.   ,  -1.   ,  -1.   , ...,  -1.   ,   0.   ,  -1.   ],
       [ -1.   ,  -1.   ,  -1.   , ...,  -1.   ,  -1.   ,   0.   ]])

In [86]:
description = """
I used the Mapquest Route API https://developer.mapquest.com/documentation/directions-api/route/post/ to complete the matrices. 
distance_matrix[i, j] contains the distance in km for going from cities[i] to cities[j], while time_matrix[i, j] contains 
the time in seconds for that route. -1 was used to fill the missing values, but I think that the API returned 0 mistakenly in a few cases,
so 0s outside of the diagonal have been set to -2
"""


distances_times = {
    "departments": dep_codes,
    "cities": cities,
    "distance_matrix": distance_matrix,
    "time_matrix": time_matrix,
    "description": description
}

In [87]:
with open(os.path.join("processed_data", "distances_times.pkl"), "wb") as f:
    pickle.dump(distances_times, f)